In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder \
    .appName("JupyterStandalone") \
    .master("spark://8fa087ac675c:7077") \
    .config("spark.executor.instances", "1") \
    .config("spark.executor.cores", "2") \
    .config("spark.executor.memory", "1g") \
    .getOrCreate()
sc = spark.sparkContext

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/06/23 12:17:51 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [26]:
rdd1=sc.parallelize([("key1",1),("key1",9),("key1",43),("key1",54),("key1",76),("key1",75),("key1",31),("key1",85),("key1",97),("key1",57),("key1",25),("key1",94),("key1",14)],3)












In [27]:
rdd1.saveAsTextFile("/home/original2")

In [28]:
def reducerfunc(x,y):
    with open("/home/m3valuesofxandy.txt","a") as file:
        file.write(f"value of x={x} and value of y={y}")
    return x+y

In [29]:
rdd1.glom().collect()

[[('key1', 1), ('key1', 9), ('key1', 43), ('key1', 54)],
 [('key1', 76), ('key1', 75), ('key1', 31), ('key1', 85)],
 [('key1', 97), ('key1', 57), ('key1', 25), ('key1', 94), ('key1', 14)]]

In [30]:
rdd2=rdd1.reduceByKey(reducerfunc)

In [31]:
rdd2.collect()

[('key1', 661)]

# output for executor /worker no 1

value of x=97 and value of y=57value of x=154 and value of y=25value of x=179 and value of y=94value of x=273 and value of y=14value of x=1 and value of y=9value of x=10 and value of y=43value of x=53 and value of y=54value of x=287 and value of y=107value of x=394 and value of y=267


we can see that partition 1 and 3 lied in worker 1 

it did sum as 

x=sumupto this but if it is first sum then x= any of the data

y= another data

it calculated sum for all partitions


so local sum

then it is shuffled across partitions so that values with same key end in same executor

now in the executor where the key has ended up 

as we can see in this case also the sum of key1 key has ended up in this cluster as well

hamlae yo verify garna sakxam mathi ko data bata


as partition ko sum 267 pani yei worker node ma aai raa xa

so yellae 3 nai oota ko sum garxa 

aani result dini vayo

Absolutely! Let's unpack **what `x` and `y` really are** in the lambda function used in `reduceByKey`.

---

### When you do:

```python
rdd.reduceByKey(lambda x, y: x + y)
```

---

### What are `x` and `y`?

* The **RDD is a collection of key-value pairs**, like:

  ```python
  [('a', 1), ('a', 2), ('b', 5), ('a', 3)]
  ```

* For each **key**, Spark wants to combine all the values.

* `reduceByKey` applies the lambda function to **pairs of values for the same key**, repeatedly, until all values for that key are combined into one.

---

### So,

* **`x` and `y` are values associated with the same key.**

* For key `'a'`, Spark will do something like:

  1. Take first two values: `x = 1`, `y = 2` → combine (`1 + 2 = 3`)
  2. Take result (`3`) and next value `3`: `x = 3`, `y = 3` → combine (`3 + 3 = 6`)

* The function (`lambda x, y: x + y`) tells how to combine two values at a time.

---

### Visual:

| Step | x | y | Result of x + y |
| ---- | - | - | --------------- |
| 1    | 1 | 2 | 3               |
| 2    | 3 | 3 | 6               |

---

### Important points:

* You can replace `lambda x, y: x + y` with any **binary function** that combines two values, like max, min, multiplication, etc.

* Spark calls this function **many times internally** to merge all values for each key.

---

### TL;DR:

* `x` and `y` are **two values from the same key’s value list**.
* The function defines how to merge those two values.
* Spark does this repeatedly until **all values for the key are merged into one**.

---

If you want, I can write a small example with print statements to show how Spark calls this function internally! Would you like that?
